In [9]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import RSIIndicator, StochasticOscillator
from ta.trend import MACD
from tqdm import tqdm

# Ticker symbols for the 12 stocks
tickers = ["FM"]

# Download historical OHLCV from 2014-01-01 to 2023-12-31
def download_data(ticker):
    data = yf.download(ticker, start="2014-01-01", end="2023-12-31")
    data = data.dropna()
    return data

stock_data = {ticker: download_data(ticker) for ticker in tqdm(tickers)}

[*********************100%***********************]  1 of 1 completed
100%|██████████| 1/1 [00:00<00:00, 11.58it/s]


In [10]:
def compute_features(df):
    df = df.copy()

    # Ensure Close, High, Low are Series (1D)
    close = df["Close"].squeeze()
    high = df["High"].squeeze()
    low = df["Low"].squeeze()
    volume = df["Volume"].squeeze()

    print(f"Close dtype: {type(close)}, shape: {close.shape}")

    # Daily return
    df["Return"] = close.pct_change()

    # 30-day rolling volatility (target)
    df["Volatility"] = df["Return"].rolling(window=30).std()

    # RSI (14 days)
    df["RSI"] = RSIIndicator(close=close, window=14).rsi()

    # Momentum (5 days)
    df["MOM"] = close - close.shift(5)

    # OBV
    df["OBV"] = (np.sign(close.diff()) * volume).fillna(0).cumsum()

    # MACD
    macd = MACD(close=close, window_slow=26, window_fast=12, window_sign=9)
    df["MACD_LINE"] = macd.macd()
    df["MACD_SIGNAL"] = macd.macd_signal()
    df["MACD_HIST"] = macd.macd_diff()

    # Stochastic Oscillator
    stoch = StochasticOscillator(high=high, low=low, close=close, window=14, smooth_window=3)
    df["STO_K"] = stoch.stoch()           # formerly %K
    df["STO_D"] = stoch.stoch_signal()    # formerly %D

    # Lagged volatilities (t-1 to t-6)
    for i in range(1, 7):
        df[f"Vol_t_{i}"] = df["Volatility"].shift(i)

    # Volatility t+1 (our target)
    df["Vol_target"] = df["Volatility"].shift(-1)

    # Drop rows with NaNs
    df = df.dropna()

    return df

In [11]:
import os
import pickle

feature_data = {}
for ticker in tqdm(tickers):
    feature_data[ticker] = compute_features(stock_data[ticker])

feature_data_path = "fm_feature_data.pkl"

if os.path.exists(feature_data_path):
    print("📦 Loading saved feature data from fm_feature_data.pkl...")
    with open(feature_data_path, "rb") as f:
        feature_data = pickle.load(f)
else:
    print("⚙️ Computing feature data...")
    feature_data = {ticker: compute_features(stock_data[ticker]) for ticker in tqdm(tickers)}
    with open(feature_data_path, "wb") as f:
        pickle.dump(feature_data, f)
    print("💾 Saved feature data to fm_feature_data.pkl")

100%|██████████| 1/1 [00:00<00:00, 71.96it/s]


Close dtype: <class 'pandas.core.series.Series'>, shape: (2516,)
⚙️ Computing feature data...


100%|██████████| 1/1 [00:00<00:00, 76.85it/s]

Close dtype: <class 'pandas.core.series.Series'>, shape: (2516,)
💾 Saved feature data to fm_feature_data.pkl


In [12]:
from arch import arch_model
import warnings

def add_garch_predictions(df, ticker=None, verbose=True):
    df = df.copy()
    returns = df["Return"].dropna().values
    preds = []
    window_size = 500
    scale_factor = 100  # recommended by arch package

    if verbose:
        print(f"\n🔍 GARCH modeling for {ticker} — total points: {len(returns)}")

    for i in range(window_size, len(returns)):
        train_window = returns[i-window_size:i] * scale_factor  # rescale

        try:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                model = arch_model(train_window, vol='Garch', p=1, q=1, dist='normal', rescale=False)
                model_fit = model.fit(disp="off")
                forecast = model_fit.forecast(horizon=1)
                pred_vol_scaled = np.sqrt(forecast.variance.values[-1][0])
                pred_vol = pred_vol_scaled / scale_factor  # unscale
        except Exception as e:
            if verbose:
                print(f"⚠️ Failed at i={i} — {e}")
            pred_vol = np.nan

        preds.append(pred_vol)

        if verbose and i % 250 == 0:
            print(f"  → Index {i} | Pred Vol (unscaled): {pred_vol:.5f}")

    full_preds = [np.nan] * window_size + preds
    df["GARCH_pred"] = full_preds

    before = len(df)
    df = df.dropna()
    after = len(df)

    if verbose:
        print(f"✅ Done {ticker} | Rows dropped: {before - after} | Final: {after} rows")

    return df

In [13]:
# === Try loading precomputed garch_data from disk ===
garch_data_path = "fm_garch_data.pkl"

if os.path.exists(garch_data_path):
    print("📦 Loading saved GARCH data from fm_garch_data.pkl...")
    with open(garch_data_path, "rb") as f:
        garch_data = pickle.load(f)
    print("✅ Loaded GARCH data successfully!")
else:
    print("⚙️ Computing GARCH data from scratch...")
    garch_data = {}
    for ticker in tickers:
        print(f"\n====================== {ticker} ======================")
        garch_data[ticker] = add_garch_predictions(feature_data[ticker], ticker=ticker)

    # Save to disk
    with open(garch_data_path, "wb") as f:
        pickle.dump(garch_data, f)
    print("💾 Saved GARCH data to fm_garch_data.pkl")

⚙️ Computing GARCH data from scratch...

====================== FM ======================

🔍 GARCH modeling for FM — total points: 2479
  → Index 500 | Pred Vol (unscaled): 0.00944
  → Index 750 | Pred Vol (unscaled): 0.00889
  → Index 1000 | Pred Vol (unscaled): 0.01412
  → Index 1250 | Pred Vol (unscaled): 0.00722
  → Index 1500 | Pred Vol (unscaled): 0.00635
  → Index 1750 | Pred Vol (unscaled): 0.00864
  → Index 2000 | Pred Vol (unscaled): 0.00828
  → Index 2250 | Pred Vol (unscaled): 0.00825
✅ Done FM | Rows dropped: 500 | Final: 1979 rows
💾 Saved GARCH data to fm_garch_data.pkl


In [14]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
import numpy as np
import warnings
warnings.filterwarnings("ignore")

def evaluate(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "RMSE": mean_squared_error(y_true, y_pred, squared=False),
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
    }

def train_ml_models_baseline(df, ticker="TICKER"):
    print(f"\n📈 Training ML models for {ticker}...")

    # Feature and target selection
    features = [
        'RSI', 'MOM', 'OBV', 'MACD_LINE', 'MACD_SIGNAL', 'MACD_HIST',
        'STO_K', 'STO_D',
        'Vol_t_1', 'Vol_t_2', 'Vol_t_3', 'Vol_t_4', 'Vol_t_5', 'Vol_t_6'
    ]

    X = df[features].copy()
    # Sanitize column names just in case LightGBM is sensitive
    X.columns = [str(col).replace("-", "_").replace("%", "PCT").replace(".", "_DOT_") for col in X.columns]

    y = df["Vol_target"]

    # Static train-test split (same as paper: 2014–2020 train, 2021–2023 test)
    split_idx = int(len(df) * 0.7)
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

    models = {
        "KNN": KNeighborsRegressor(),
        "AdaBoost": AdaBoostRegressor(),
        "CatBoost": CatBoostRegressor(verbose=0),
        #"LightGBM": LGBMRegressor(),
        "XGBoost": XGBRegressor(verbosity=0),
        "RandomForest": RandomForestRegressor()
    }

    results = {}

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        metrics = evaluate(y_test, y_pred)
        results[name] = metrics
        print(f"✅ {name} — R²: {metrics['R2']:.4f}, RMSE: {metrics['RMSE']:.4f}, MAE: {metrics['MAE']:.4f}")

    return results

/shared/home/ads066/.local/lib/python3.11/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


In [15]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def evaluate(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    return {
        "R2": r2_score(y_true, y_pred),
        "RMSE": np.sqrt(mse),
        "MSE": mse,
        "MAE": mean_absolute_error(y_true, y_pred),
    }

def train_all_stocks_ml_baseline(garch_data_dict, results_path="fm_ml_baseline_results.csv"):
    # Check if results already exist
    if os.path.exists(results_path):
        print(f"📦 Loading existing results from {results_path}...")
        return pd.read_csv(results_path)

    final_results = []

    for ticker, df in garch_data_dict.items():
        print(f"\n================= {ticker} =================")
        results = train_ml_models_baseline(df, ticker=ticker)

        for model_name, metrics in results.items():
            final_results.append({
                "Stock": ticker,
                "Model": model_name,
                "R2": round(metrics["R2"], 4),
                "RMSE": round(metrics["RMSE"], 4),
                "MSE": round(metrics["MSE"], 6),
                "MAE": round(metrics["MAE"], 4),
            })

    # Save results
    results_df = pd.DataFrame(final_results)
    results_df.to_csv(results_path, index=False)
    print(f"💾 Saved results to {results_path}")

    return results_df

# Run training or load existing results
ml_all_results = train_all_stocks_ml_baseline(garch_data)
ml_all_results_sorted = ml_all_results.sort_values(by="R2", ascending=False)
display(ml_all_results_sorted)


================= FM =================

📈 Training ML models for FM...
✅ KNN — R²: -2.0719, RMSE: 0.0031, MAE: 0.0024
✅ AdaBoost — R²: -2.6954, RMSE: 0.0034, MAE: 0.0016
✅ CatBoost — R²: -0.2262, RMSE: 0.0020, MAE: 0.0010
✅ XGBoost — R²: -0.1657, RMSE: 0.0019, MAE: 0.0007
✅ RandomForest — R²: -0.1574, RMSE: 0.0019, MAE: 0.0008
💾 Saved results to fm_ml_baseline_results.csv


,Stock,Model,R2,RMSE,MSE,MAE
4,FM,RandomForest,-0.1574,0.0019,0.000004,0.0008
3,FM,XGBoost,-0.1657,0.0019,0.000004,0.0007
2,FM,CatBoost,-0.2262,0.0020,0.000004,0.0010
0,FM,KNN,-2.0719,0.0031,0.000010,0.0024
1,FM,AdaBoost,-2.6954,0.0034,0.000012,0.0016


In [16]:
from arch import arch_model
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def evaluate_series(y_true, y_pred):
    return {
        "R2": r2_score(y_true, y_pred),
        "RMSE": mean_squared_error(y_true, y_pred, squared=False),
        "MSE": mean_squared_error(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
    }

def forecast_volatility_arch(df, model_type="GARCH", ticker="TICKER", verbose=True):
    df = df.copy()

    # Flatten columns if MultiIndex
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ['_'.join([str(i) for i in col if i]) for col in df.columns]

    if "Vol_target" not in df.columns or "Return" not in df.columns:
        raise KeyError(f"Missing 'Vol_target' or 'Return' in {ticker}")

    returns = df["Return"].dropna().values
    preds = []
    window_size = 500
    scale_factor = 100  # fix for scale warning

    if verbose:
        print(f"\n🔮 Running {model_type} for {ticker}...")

    for i in range(window_size, len(returns)):
        train_window = returns[i-window_size:i] * scale_factor

        try:
            if model_type == "GARCH":
                model = arch_model(train_window, vol='GARCH', p=1, q=1, dist='normal', rescale=False)
            elif model_type == "GJR":
                model = arch_model(train_window, vol='GARCH', p=1, o=1, q=1, dist='normal', rescale=False)
            elif model_type == "EGARCH":
                model = arch_model(train_window, vol='EGARCH', p=1, q=1, dist='normal', rescale=False)
            else:
                raise ValueError("Invalid model_type")

            model_fit = model.fit(disp="off")
            forecast = model_fit.forecast(horizon=1)
            pred_vol = np.sqrt(forecast.variance.values[-1][0]) / scale_factor

        except Exception as e:
            if verbose:
                print(f"⚠️ {model_type} failed at index {i}: {e}")
            pred_vol = np.nan

        preds.append(pred_vol)

        if verbose and i % 250 == 0:
            print(f"  → {model_type} | index {i} | vol: {pred_vol:.5f}")

    df[f"{model_type}_pred"] = [np.nan] * window_size + preds
    df = df.dropna(subset=["Vol_target", f"{model_type}_pred"])

    metrics = evaluate_series(df["Vol_target"], df[f"{model_type}_pred"])
    if verbose:
        print(f"✅ {model_type} for {ticker} — R²: {metrics['R2']:.4f}, RMSE: {metrics['RMSE']:.4f}, MAE: {metrics['MAE']:.4f}")

    return df, metrics

In [17]:
import os
import pandas as pd

def evaluate_all_series_models(garch_data_dict, results_path="fm_ts_model_results.csv"):
    # If results already exist, load them
    if os.path.exists(results_path):
        print(f"📦 Loading saved time series results from {results_path}...")
        return pd.read_csv(results_path)

    results = []

    for ticker, df in garch_data_dict.items():
        for model_type in ["GARCH", "GJR", "EGARCH"]:
            print(f"\n================= {ticker} - {model_type} =================")
            try:
                _, metrics = forecast_volatility_arch(df, model_type=model_type, ticker=ticker, verbose=True)
                results.append({
                    "Stock": ticker,
                    "Model": model_type,
                    "R2": round(metrics["R2"], 4),
                    "RMSE": round(metrics["RMSE"], 4),
                    "MSE": round(metrics["MSE"], 6),
                    "MAE": round(metrics["MAE"], 4),
                })
            except Exception as e:
                print(f"⚠️ Skipping {ticker} - {model_type}: {e}")

    df_results = pd.DataFrame(results)
    df_results.to_csv(results_path, index=False)
    print(f"💾 Saved time series model results to {results_path}")

    return df_results

ts_model_results = evaluate_all_series_models(garch_data)
ts_model_results_sorted = ts_model_results.sort_values(by="R2", ascending=False)
display(ts_model_results_sorted)


================= FM - GARCH =================

🔮 Running GARCH for FM...
  → GARCH | index 500 | vol: 0.01412
  → GARCH | index 750 | vol: 0.00722
  → GARCH | index 1000 | vol: 0.00635
  → GARCH | index 1250 | vol: 0.00864
  → GARCH | index 1500 | vol: 0.00828
  → GARCH | index 1750 | vol: 0.00825
✅ GARCH for FM — R²: 0.6259, RMSE: 0.0034, MAE: 0.0018

================= FM - GJR =================

🔮 Running GJR for FM...
  → GJR | index 500 | vol: 0.00718
  → GJR | index 750 | vol: 0.00742
  → GJR | index 1000 | vol: 0.00619
  → GJR | index 1250 | vol: 0.00869
  → GJR | index 1500 | vol: 0.00875
  → GJR | index 1750 | vol: 0.00791
✅ GJR for FM — R²: 0.4287, RMSE: 0.0042, MAE: 0.0020

================= FM - EGARCH =================

🔮 Running EGARCH for FM...
  → EGARCH | index 500 | vol: 0.01034
  → EGARCH | index 750 | vol: 0.00684


/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/p

  → EGARCH | index 1000 | vol: 0.00000


/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


  → EGARCH | index 1250 | vol: 0.00879
  → EGARCH | index 1500 | vol: 0.00868
  → EGARCH | index 1750 | vol: 0.00829


/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


✅ EGARCH for FM — R²: 0.4136, RMSE: 0.0042, MAE: 0.0020
💾 Saved time series model results to fm_ts_model_results.csv


,Stock,Model,R2,RMSE,MSE,MAE
0,FM,GARCH,0.6259,0.0034,0.000011,0.0018
1,FM,GJR,0.4287,0.0042,0.000017,0.0020
2,FM,EGARCH,0.4136,0.0042,0.000018,0.0020


In [18]:
def train_fusion_model(df, model_name, ts_feature="GARCH_pred", ticker="TICKER"):
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
    from sklearn.neighbors import KNeighborsRegressor
    from catboost import CatBoostRegressor
    from lightgbm import LGBMRegressor
    from xgboost import XGBRegressor

    models = {
        "KNN": KNeighborsRegressor(),
        "AdaBoost": AdaBoostRegressor(),
        "CatBoost": CatBoostRegressor(verbose=0),
        "XGBoost": XGBRegressor(verbosity=0),
        "RandomForest": RandomForestRegressor()
    }

    if model_name not in models:
        raise ValueError(f"Model '{model_name}' not recognized.")

    df = df.copy()

    # Flatten if needed
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ['_'.join([str(i) for i in col if i]) for col in df.columns]

    if ts_feature not in df.columns:
        raise ValueError(f"'{ts_feature}' not found in DataFrame for {ticker}")

    feature_cols = [
        'RSI', 'MOM', 'OBV', 'MACD_LINE', 'MACD_SIGNAL', 'MACD_HIST',
        'STO_K', 'STO_D', 'Vol_t_1', 'Vol_t_2', 'Vol_t_3',
        'Vol_t_4', 'Vol_t_5', 'Vol_t_6', ts_feature
    ]

    X = df[feature_cols].copy()
    y = df["Vol_target"]

    split_idx = int(len(df) * 0.7)
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

    model = models[model_name]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    metrics = evaluate_series(y_test, y_pred)

    print(f"✅ {model_name} + {ts_feature} for {ticker} — R²: {metrics['R2']:.4f}, RMSE: {metrics['RMSE']:.4f}, MAE: {metrics['MAE']:.4f}")

    return metrics

In [19]:
import os
import pandas as pd

# Reuse existing forecast function
for ticker in tqdm(garch_data.keys()):
    for model_type in ["GJR", "EGARCH"]:
        print(f"\n📈 Adding {model_type}_pred to {ticker}...")
        df = garch_data[ticker]

        try:
            df, _ = forecast_volatility_arch(df, model_type=model_type, ticker=ticker, verbose=False)
            garch_data[ticker] = df  # Update with new column
        except Exception as e:
            print(f"⚠️ {model_type} failed for {ticker}: {e}")

def train_all_fusion_models(garch_data_dict, results_path="fm_fusion_model_results.csv"):
    # Load existing results if file exists
    if os.path.exists(results_path):
        print(f"📦 Loading saved fusion model results from {results_path}...")
        return pd.read_csv(results_path)

    results = []

    for ticker, df in garch_data_dict.items():
        for ts_feature in ["GARCH_pred", "GJR_pred", "EGARCH_pred"]:
            if ts_feature not in df.columns:
                print(f"⚠️ Skipping {ticker} - missing {ts_feature}")
                continue

            for model_name in ["RandomForest", "XGBoost", "CatBoost", "AdaBoost", "KNN"]:
                try:
                    metrics = train_fusion_model(df, model_name, ts_feature=ts_feature, ticker=ticker)
                    results.append({
                        "Stock": ticker,
                        "Fusion_Model": f"{ts_feature}+{model_name}",
                        "R2": round(metrics["R2"], 4),
                        "RMSE": round(metrics["RMSE"], 4),
                        "MSE": round(metrics["MSE"], 6),
                        "MAE": round(metrics["MAE"], 4),
                    })
                except Exception as e:
                    print(f"⚠️ {ticker} {ts_feature}+{model_name} failed: {e}")

    fusion_df = pd.DataFrame(results)
    fusion_df.to_csv(results_path, index=False)
    print(f"💾 Saved fusion model results to {results_path}")

    return fusion_df

fusion_results_df = train_all_fusion_models(garch_data)
fusion_results_sorted = fusion_results_df.sort_values(by="R2", ascending=False)
display(fusion_results_sorted)

  0%|          | 0/1 [00:00<?, ?it/s]


📈 Adding GJR_pred to FM...

📈 Adding EGARCH_pred to FM...


/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/shared/home/ads066/.local/lib/python3.11/site-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
100%|██████████| 1/1 [00:34<00:00, 34.89s/it]


✅ RandomForest + GARCH_pred for FM — R²: 0.6650, RMSE: 0.0009, MAE: 0.0006
✅ XGBoost + GARCH_pred for FM — R²: 0.7565, RMSE: 0.0008, MAE: 0.0006
✅ CatBoost + GARCH_pred for FM — R²: 0.8144, RMSE: 0.0007, MAE: 0.0005
✅ AdaBoost + GARCH_pred for FM — R²: 0.5784, RMSE: 0.0010, MAE: 0.0008
✅ KNN + GARCH_pred for FM — R²: -3.5604, RMSE: 0.0033, MAE: 0.0023
✅ RandomForest + GJR_pred for FM — R²: 0.5950, RMSE: 0.0010, MAE: 0.0006
✅ XGBoost + GJR_pred for FM — R²: 0.7556, RMSE: 0.0008, MAE: 0.0006
✅ CatBoost + GJR_pred for FM — R²: 0.8317, RMSE: 0.0006, MAE: 0.0005
✅ AdaBoost + GJR_pred for FM — R²: 0.5368, RMSE: 0.0011, MAE: 0.0009
✅ KNN + GJR_pred for FM — R²: -3.5604, RMSE: 0.0033, MAE: 0.0023
✅ RandomForest + EGARCH_pred for FM — R²: 0.6997, RMSE: 0.0009, MAE: 0.0006
✅ XGBoost + EGARCH_pred for FM — R²: 0.7643, RMSE: 0.0008, MAE: 0.0006
✅ CatBoost + EGARCH_pred for FM — R²: 0.8233, RMSE: 0.0007, MAE: 0.0005
✅ AdaBoost + EGARCH_pred for FM — R²: 0.6812, RMSE: 0.0009, MAE: 0.0007
✅ KNN + EGA

,Stock,Fusion_Model,R2,RMSE,MSE,MAE
7,FM,GJR_pred+CatBoost,0.8317,0.0006,0.000000,0.0005
12,FM,EGARCH_pred+CatBoost,0.8233,0.0007,0.000000,0.0005
2,FM,GARCH_pred+CatBoost,0.8144,0.0007,0.000000,0.0005
11,FM,EGARCH_pred+XGBoost,0.7643,0.0008,0.000001,0.0006
1,FM,GARCH_pred+XGBoost,0.7565,0.0008,0.000001,0.0006
6,FM,GJR_pred+XGBoost,0.7556,0.0008,0.000001,0.0006
10,FM,EGARCH_pred+RandomForest,0.6997,0.0009,0.000001,0.0006
13,FM,EGARCH_pred+AdaBoost,0.6812,0.0009,0.000001,0.0007
0,FM,GARCH_pred+RandomForest,0.6650,0.0009,0.000001,0.0006
5,FM,GJR_pred+RandomForest,0.5950,0.0010,0.000001,0.0006
